# Environment Setup

In [1]:
import nest_asyncio
from dotenv import find_dotenv, load_dotenv
from pprint import pprint

nest_asyncio.apply()
load_dotenv(find_dotenv())


True

# Output

The `output` is wrapped in `AgentRunResult` or `StreamedRunResult` so you can access other data like `usage` of the run and `message_history`

Both `AgentRunResult` and `StreamedRunResult` are generic in the data they wrap, so typing information about the data returned by the agent is preserved.

## Usage

In [2]:
from pydantic import BaseModel

from pydantic_ai import Agent


class CityLocation(BaseModel):
    city: str
    country: str


agent = Agent("google-gla:gemini-1.5-flash", output_type=CityLocation)
result = agent.run_sync("Where were the olympics held in 2012?")
pprint(result.output)

CityLocation(city='London', country='UK')


In [3]:
print(result.usage())

Usage(requests=1, request_tokens=27, response_tokens=7, total_tokens=34, details=None)


## Output Data

### Structured Outputs

Structured outputs (like tools) use Pydantic to build the JSON schema used for the tool, and to validate the data returned by the model.

In [23]:
from typing import Union

from pydantic import BaseModel

from pydantic_ai import Agent

class Box(BaseModel):
    width: int
    height: int
    depth: int
    units: str

agent: Agent[None, Union[Box, str]] = Agent(
    "google-gla:gemini-1.5-flash",
    output_type=Union[Box, str],  # type: ignore
    system_prompt=(
        "Extract me the dimensions of a box, "
        "if you can't extract all data, ask the user to try again."
    ),
)

In [26]:
result = agent.run_sync("The box is 10x20x30")
print(result.output)

width=30 height=20 depth=10 units='x'


In [27]:
result = agent.run_sync("The box is 10x20x30 cm")
print(result.output)

width=10 height=20 depth=30 units='cm'


### Using a Union Return Type

Here's an example of using a union return type which registers multiple tools, and wraps non-object schemas in an object

In [ ]:
from typing import Union

agent: Agent[None, Union[list[str], list[int]]] = Agent(
    "google-gla:gemini-1.5-flash-8b",
    output_type=Union[list[str], list[int]],  # type: ignore
    system_prompt="Extract either colors or sizes from the shapes provided.",
)

In [6]:
result = agent.run_sync("red square, blue circle, green triangle")
print(result.output)


['red', 'blue', 'green']


In [7]:
result = agent.run_sync("square size 10, circle size 20, triangle size 30")
print(result.output)


[10, 20, 30]


### Output Validator Functions

Some validation is inconvenient or impossible to do in Pydantic validators, in particular when the validation requires IO and is asynchronous. PydanticAI provides a way to add validation functions via the `agent.output_validator` decorator.

## Streamed Results

**There two main challenges with streamed results:**

1. Validating structured responses before they're complete, this is achieved by "partial validation" which was recently added to Pydantic.
2. When receiving a response, we don't know if it's the final response without starting to stream it and peeking at the content. PydanticAI streams just enough of the response to sniff out if it's a tool call or an output, then streams the whole thing and calls tools, or returns the stream as a `StreamedRunResult`.

### Streaming Text

In [31]:
agent = Agent("google-gla:gemini-1.5-flash")


async def stream_text():
    async with agent.run_stream('Where does "hello world" come from?') as result:
        async for message in result.stream_text():
            print(message)


In [32]:
import asyncio
asyncio.run(stream_text())

The "Hello, world!" program originates from Brian Kernighan's 1
The "Hello, world!" program originates from Brian Kernighan's 1972 book, "A Tutorial Introduction to the Language B".  While
The "Hello, world!" program originates from Brian Kernighan's 1972 book, "A Tutorial Introduction to the Language B".  While not explicitly called "Hello, world!" in that version, it presented a similar program that printed "hello, world" to demonstrate a basic "print" function
The "Hello, world!" program originates from Brian Kernighan's 1972 book, "A Tutorial Introduction to the Language B".  While not explicitly called "Hello, world!" in that version, it presented a similar program that printed "hello, world" to demonstrate a basic "print" function in the programming language B (a precursor to C).

While Kernighan didn't coin the phrase itself, his example in the B tutorial is widely considered the genesis of the now-ubiquitous "Hello, world!" program.  It became a standard introductory programmin

**We can also stream text as deltas rather than the entire text in each item:**

In [33]:
async def stream_delta():
    async with agent.run_stream('Where does "hello world" come from?') as result:
        async for message in result.stream_text(delta=True):
            print(message)

In [34]:
asyncio.run(stream_delta())

The "Hello, world!" program originates from Brian Kernighan's 1
972 document, "A Tutorial Introduction to the Language B."  While
 not the very first example of a simple program printing text, it's the one that became incredibly popular and has since been widely adopted as the canonical introductory program
 in almost every programming language tutorial.  Its simplicity and immediate visual result make it ideal for demonstrating the basic syntax and functionality of a new language.  Before this, similar examples existed, but Kernighan's version cemented its place in programming history.



>Output message not included in **`messages`**:\
>The final output message will **NOT** be added to result messages if you use `.stream_text(delta=True),` see [Messages and chat history](https://ai.pydantic.dev/message-history/) for more information.

### Streaming Structured Output

Not all types are supported with partial validation in Pydantic, see [pydantic/pydantic#10748](https://github.com/pydantic/pydantic/pull/10748), generally for model-like structures it's currently best to use `TypeDict`.

In [46]:
from datetime import date

from typing_extensions import TypedDict

from pydantic_ai import Agent


class UserProfile(TypedDict, total=False):
    name: str
    dob: date
    bio: str


agent = Agent(
    "google-gla:gemini-2.5-flash-preview-04-17",
    output_type=UserProfile,
    system_prompt="Extract a user profile from the input",
)


async def stream_struct():
    user_input = "My name is Ben, I was born on January 28th 1990, I like the chain the dog and the pyramid."
    async with agent.run_stream(user_input) as result:
        async for profile in result.stream():
            print(profile)


In [47]:
asyncio.run(stream_struct())

{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'I like the chain the dog and the pyramid'}
{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'I like the chain the dog and the pyramid'}


If you want fine-grained control of validation, particularly catching validation errors, you can use the following pattern:

In [51]:
from datetime import date

from pydantic import ValidationError
from typing_extensions import TypedDict

from pydantic_ai import Agent


class UserProfile(TypedDict, total=False):
    name: str
    dob: date
    bio: str


agent = Agent("google-gla:gemini-2.0-pro-exp-02-05", output_type=UserProfile)
user_input = "My name is Ben, I was born on January 28th 1990, I like the chain the dog and the pyramid."

In [52]:
async def stream_with_validation():
    
    async with agent.run_stream(user_input) as result:
        async for message, last in result.stream_structured(debounce_by=0.01):
            try:
                profile = await result.validate_structured_output(
                    message,
                    allow_partial=not last,
                )
            except ValidationError:
                continue
            print(profile)


In [53]:
asyncio.run(stream_with_validation())

{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'I like the chain the dog and the pyramid'}
{'name': 'Ben', 'dob': datetime.date(1990, 1, 28), 'bio': 'I like the chain the dog and the pyramid'}
